In [2]:
import pandas as pd
import os
from datasets import Dataset
from transformers import AutoTokenizer

current_dir = os.getcwd()
print(current_dir)
if current_dir.endswith("week5"):
    os.chdir("../..")
    print(os.getcwd())
else:
    print("current dir", current_dir)


/opt/miniconda3/envs/ml/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course/code/week5
/Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course


In [3]:

ds_train = pd.read_parquet("dataset/train_df.parquet")
ds_val = pd.read_parquet("dataset/val_df.parquet")

ds_train_inlang = ds_train[ds_train['answer_inlang'].notna()]
ds_val_inlang = ds_val[ds_val['answer_inlang'].notna()]
ds_train_en = ds_train
ds_val_en = ds_val

In [8]:
#basic dataset
ds_train_inlang.drop(columns=[col for col in ds_train_inlang.columns if col not in ["question", "context", "answer_inlang"]], inplace=True)
ds_val_inlang.drop(columns=[col for col in ds_val_inlang.columns if col not in ["question", "context", "answer_inlang"]], inplace=True)
ds_train_en.drop(columns=[col for col in ds_train_en.columns if col not in ["question", "context", "answer"]], inplace=True)
ds_val_en.drop(columns=[col for col in ds_val_en.columns if col not in ["question", "context", "answer"]], inplace=True)



ds_train_inlang.to_json("code/week5/data/finetune_base_inlang.jsonl", orient="records", lines=True)
ds_val_inlang.to_json("code/week5/data/finetune_base_inlang_val.jsonl", orient="records", lines=True)
ds_train_en.to_json("code/week5/data/finetune_base_en.jsonl", orient="records", lines=True)
ds_val_en.to_json("code/week5/data/finetune_base_en_val.jsonl", orient="records", lines=True)



/var/folders/mw/rr00tj6s1tz9vtmb2mgqnfbc0000gn/T/ipykernel_39084/707296779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_train_inlang.drop(columns=[col for col in ds_train_inlang.columns if col not in ["question", "context", "answer_inlang"]], inplace=True)
/var/folders/mw/rr00tj6s1tz9vtmb2mgqnfbc0000gn/T/ipykernel_39084/707296779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_val_inlang.drop(columns=[col for col in ds_val_inlang.columns if col not in ["question", "context", "answer_inlang"]], inplace=True)


In [4]:
from typing import Any, List, Optional
import json

remove_sentence = """Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024"""


SYSTEM_MESSAGE = """Given the users context(in english) and question(in either finnish, russian or japanese), answer the question(which is a substring of the context) in the same language as the question(either finnish, russian or japanese)."""

def construct_prompt(
    tokenizer : AutoTokenizer,
    question : str,
    context : str,
    answer : Optional[str] = None,
    tokenize : bool = False
) -> str:
    
    messages = [
        {"role": "system", "content": f"{SYSTEM_MESSAGE}"},
        {"role": "user", "content": f"Question: {question}\nContext: {context}"},
    ]
    if answer:
        messages.append({"role": "assistant", "content": f"{answer}"})
    prompt = tokenizer.apply_chat_template(messages, tokenize=tokenize, add_generation_prompt=False, format="chatml")
    return prompt


import re
from datasets import Dataset    

def prompt_to_segments(
    prompt: str,
) -> List[dict[str, Any]]:
    """
    Convert a prompt to a list of segments.
    """
    
    middle_text = 'assistant<\|end_header_id\|>'
    end_text = '<\|eot_id\|>'
    pattern = r'(.*?)assistant<\|end_header_id\|>(.*?)<\|eot_id\|>'
    match = re.search(pattern, prompt, re.DOTALL)
    chunks = [
        {"text": match.group(1), "label": "false"},
        {"text": middle_text, "label": "false"},
        {"text": match.group(2), "label": "true"},
        {"text": end_text, "label": "false"}
    ]
    return chunks

def construct_input_output_from_df(
    df: pd.DataFrame,
    tokenizer : AutoTokenizer,
    write_path : str,
    answer_key : str = "answer_inlang",
):
    df['prompt_str'] = df.apply(
        lambda x: 
            construct_prompt(
                tokenizer, x['question'], x['context'], x[answer_key], tokenize=True
            ), axis=1
        )
    
    with open(write_path, 'w') as f:
        for _, row in df.iterrows():
            chatml_format = {
                "input": row['prompt_str'],
                "output": row[answer_key]
            }
            f.write(json.dumps(chatml_format) + '\n')    
    
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

def create_chatml_dataset(
    df : pd.DataFrame,
    answer_key : str = "answer_inlang",
):
    
    df['system_message'] = SYSTEM_MESSAGE
    df['conversation'] = df.apply(
        lambda x: [
            {"from": "system", "content": f"{SYSTEM_MESSAGE}"},
            {"from": "user", "content": f"Question: {x['question']}\nContext: {x['context']}"},
            {"from": "assistant", "content": f"{x[answer_key]}"}
        ], axis=1
    )

    df = df.drop(columns=[col for col in df.columns if col not in ["system_message", "conversation"]])
    dataset = Dataset.from_pandas(df)
    return dataset

def create_alpaca_dataset(
    df: pd.DataFrame,
    answer_key: str = "answer_inlang",
):
    df['instruction'] = df.apply(
        lambda x: f"{SYSTEM_MESSAGE}", axis=1
    )
    df['input'] = df.apply(
        lambda x: f"Question: {x['question']}\nContext: {x['context']}", axis=1
    )
    df['output'] = df[answer_key]
    dataset = Dataset.from_pandas(df)
    return dataset
    
    

ds_train_chatml = create_alpaca_dataset(ds_train_inlang, answer_key="answer_inlang")
ds_val_chatml = create_alpaca_dataset(ds_val_inlang, answer_key="answer_inlang")
ds_train_chatml_en = create_alpaca_dataset(ds_train_en, answer_key="answer")
ds_val_chatml_en = create_alpaca_dataset(ds_val_en, answer_key="answer")


/var/folders/mw/rr00tj6s1tz9vtmb2mgqnfbc0000gn/T/ipykernel_39084/1200584575.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['instruction'] = df.apply(
/var/folders/mw/rr00tj6s1tz9vtmb2mgqnfbc0000gn/T/ipykernel_39084/1200584575.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['input'] = df.apply(
/var/folders/mw/rr00tj6s1tz9vtmb2mgqnfbc0000gn/T/ipykernel_39084/1200584575.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [5]:
os.getcwd()


'/Users/HansPeter/Dev/ku_local/nlp_course/src/nlp_course'

In [6]:

ds_train_chatml.to_json("code/week5/data/finetune_alpaca_inlang.jsonl", orient="records", lines=True)
ds_train_chatml_en.to_json("code/week5/data/finetune_alpaca_en.jsonl", orient="records", lines=True)
ds_val_chatml.to_json("code/week5/data/finetune_alpaca_inlang_val.jsonl", orient="records", lines=True)
ds_val_chatml_en.to_json("code/week5/data/finetune_alpaca_en_val.jsonl", orient="records", lines=True)


Creating json from Arrow format: 100%|██████████| 2/2 [00:00<00:00,  3.10ba/s]


95670836